<a href="https://colab.research.google.com/github/demelin/ai_reimplementations/blob/main/dialoue_continuation_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import os
import json
import torch
import pickle
import random
import numpy as np

from tqdm import trange, tqdm
from torch.utils.data import (
    Dataset, DataLoader, RandomSampler, SequentialSampler)
from transformers import (
    AutoConfig, AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup, get_constant_schedule)
from datasets import load_dataset

In [ ]:
# Define experiment configuration


class Config(object):

  """ Contains default settings for model training and evaluation. """

  def __init__(
      self,
      max_seq_len=256,
      train_batch_size=32,
      eval_batch_size=32,
      gradient_accumulation_steps=4,
      num_train_steps=1000,
      eval_after_steps=50,
      patience=3,
      learning_rate=1e-5,
      weight_decay=0.01,
      adam_epsilon=1e-8,
      max_grad_norm=1.0,
      random_seed=42,
      logging_steps=25,
      use_cpu=False,
      checkpoint="facebookAI/roberta-base",
      dataset="daily_dialog"
      ):

    self.max_seq_len = max_seq_len
    self.train_batch_size = train_batch_size
    self.eval_batch_size = eval_batch_size
    self.gradient_accumulation_steps = gradient_accumulation_steps
    self.num_train_steps = num_train_steps
    self.learning_rate = learning_rate
    self.eval_after_steps = eval_after_steps
    self.patience = patience
    self.weight_decay = weight_decay
    self.adam_epsilon = adam_epsilon
    self.max_grad_norm = max_grad_norm

    self.use_cpu = use_cpu
    self.checkpoint = checkpoint
    self.dataset = dataset
    self.random_seed = random_seed
    self.logging_steps = logging_steps

In [ ]:
class NewDataset(Dataset):

    """ Dataset object for model training. Expected as input by DataLoader. """

    def __init__(self, input_ids, attention_masks, labels):

        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):

        return len(self.labels)

    def __getitem__(self, item_id):

        input_ids = self.input_ids[item_id]
        attention_mask = self.attention_masks[item_id]
        label = self.labels[item_id]
        sample = {
            "input_ids": input_ids,
            "labels": label,
            "attention_mask": attention_mask,
            }

        return sample

In [ ]:
# Prepare data for training / evaluation / testing

class DataProcessor(object):

  """ Prepares data for use with the model and returs
    data + DataLoader objects. """

  def __init__(self, config, tokenizer):

    self.config = config
    self.tokenizer = tokenizer
    self.labels_list = [0, 1]
    self.num_labels = len(self.labels_list)

    # Load data
    self.train_data, self.eval_data, self.test_data = self._get_raw_data()
    # Transorm data
    self.train_data = self._transform_data(self.train_data)
    self.eval_data = self._transform_data(self.eval_data)
    self.test_data = self._transform_data(self.test_data, is_test=True)
    # Embed training and valiadtion data
    train_samples = self.encode_truncate_pad(self.train_data)
    eval_samples = self.encode_truncate_pad(self.eval_data)
    # Create Datasets
    train_dataset =  self._make_dataset(train_samples)
    eval_dataset =  self._make_dataset(eval_samples)

    # Create DataLoaders
    # Use a random sampler for training
    train_sampler = RandomSampler(train_dataset)
    self.train_dataloader = DataLoader(train_dataset,
                                       sampler=train_sampler,
                                       batch_size=self.config.train_batch_size)

    # Use a sequential sampler for evaluation for consistency
    eval_sampler = SequentialSampler(eval_dataset)
    self.eval_dataloader = DataLoader(eval_dataset,
                                      sampler=eval_sampler,
                                      batch_size=self.config.eval_batch_size)


  def _get_raw_data(self):

    """ Downloads datasets from HuggingFace Datasets;
      Assumes use of the DailyDialog dataset, may require adjustment for other
      datasets. """

    print("Dowloading data ...")
    # For prototyping purposes, we only look at a subset of the data
    train_data = load_dataset(
        self.config.dataset, split="train")["dialog"]
    eval_data = load_dataset(
        self.config.dataset, split="validation")[:500]["dialog"]
    test_data = load_dataset(
        self.config.dataset, split="test")["dialog"]

    return train_data, eval_data, test_data


  def _transform_data(self, dialog_list, num_distractors=1, is_test=False):

    """ Transforms available dialogs for the dialog-completion task. """

    print("Transforming data ...")
    # Declare dialogue participant "tags"; customizable / optional
    person_tags = ["Person 1", "Person 2"]

    # dialog_list is a list of lists, with the final element in each list
    # representing the target continuation
    # 1. Split dialogues into context and target
    # 2. Sample distractor continuations form other dialogs for each dialog
    # 3. Create model inputs by paring contexts with continuations and assigning
    # a label to each sample

    # Note: Here the distractors are selected at random, but we can exploit
    # semantic similarity and other methods to "mine" hard distractors for
    # more robust models

    # Obtain true samples
    true_samples = list()
    distractor_bank = set()
    for dlg in dialog_list:
      # Add person tags to the context sentences (may not be neccessary?)
      context_text = dlg[:-1] + [" "]
      aug_context_text = list()
      for turn_id, turn in enumerate(context_text):
        turn = "{:s}: {}".format(person_tags[turn_id % len(person_tags)],
                                 turn.strip())

        aug_context_text.append(turn)

      new_sample = {"context": ' '.join(aug_context_text),
                    "target": dlg[-1],
                    "label": 1}
      true_samples.append(new_sample)
      distractor_bank.add(new_sample["target"])

    # Get negative samples for training and eval
    if not is_test:
      # Obtain false samples
      false_samples = list()
      distractor_bank = list(distractor_bank)
      for ts in true_samples:
        for _ in range(num_distractors):
          sampled_distractor = random.choice(distractor_bank)
          # make sure the distractor is different from the true target
          while sampled_distractor == ts["target"]:
            sampled_distractor = random.choice(distractor_bank)
          false_samples.append({"context": ts["context"],
                                "target": sampled_distractor,
                                "label": 0})

        # Combine and shuffle
        all_samples = true_samples + false_samples

    else:
      # Test data is created differently, by pairing each context with N
      # (e.g. 5 here) distractors; in 50% of cases correct continuation is
      # replaced with another distractor

      random.uniform(0, 1)

      # Create test samples
      all_samples = list()
      distractor_bank = list(distractor_bank)
      for ts in true_samples:
        keep_true = random.uniform(0., 1.)
        if keep_true < 0.5:
          # Keep the true continuation
          target_list = [ts["target"]]
          num_distractors = 5
        else:
          # Replace true continuation with distractor
          target_list = list()
          num_distractors = 6

        for _ in range(num_distractors):
          sampled_distractor = random.choice(distractor_bank)
          while sampled_distractor == ts["target"] or sampled_distractor in target_list:
            sampled_distractor = random.choice(distractor_bank)
          target_list.append(sampled_distractor)

        # Get ID of correct continuation
        if keep_true < 0.5:
          label = ts["target"]
        else:
          label = -1

        all_samples.append({"context": ts["context"],
                            "targets": target_list,
                            "label": label})

    return all_samples


  def encode_truncate_pad(self, dialog_samples):
      """ Encodes, truncates, and pads the individual samples. """

      # RoBERTa specific tokenizer properties for smart-ish truncation
      num_special_tokens = 4  # 1 * <s> + 3 * </s>
      max_sample_len = self.config.max_seq_len - num_special_tokens

      # Truncation strategy: Only runcate the context, by truncating the
      # beginning of the context rather than its end, as the context end is
      # likely to be more predictive of the dialog continuation
      print("Encoding data ...")
      encoded_samples = list()
      for ds in dialog_samples:
        # Encode context and target
        context_ids = self.tokenizer.encode(ds["context"], add_special_tokens=False)
        target_ids = self.tokenizer.encode(ds["target"], add_special_tokens=False)

        # Skip sample if the response is too long
        if len(target_ids) >= (max_sample_len - 100):
          continue

        max_context_len = max_sample_len - len(target_ids)
        # Truncate context
        truncated_context_ids = context_ids[
            max(0, len(context_ids) - max_context_len):]
        sample_ids = [self.tokenizer.bos_token_id] + truncated_context_ids + [
            self.tokenizer.sep_token_id] * 2 + target_ids
        # Create attention mask
        attention_mask = [1] * len(sample_ids)
        # Pad sequences to max_seq_len if necessary
        pad_size = self.config.max_seq_len - len(sample_ids)
        if pad_size > 0:
            sample_ids += [self.tokenizer.pad_token_id] * pad_size
            attention_mask += [0] * pad_size

        # Collect
        encoded_samples.append({"input_ids": sample_ids,
                                "attention_mask": attention_mask,
                                "label": ds["label"]})

      return encoded_samples


  def _make_dataset(self, encoded_samples):

    """ Converts encoded sample into a format usable by the PLM. """

    # Convert to tensors
    feature_dict = {
      "input_ids": torch.tensor(
          [s["input_ids"] for s in encoded_samples], dtype=torch.long),
      "attention_masks": torch.tensor(
          [s["attention_mask"] for s in encoded_samples], dtype=torch.long),
      "labels": torch.tensor(
          [s["label"] for s in encoded_samples], dtype=torch.long)
      }

    # Create DataSet
    return NewDataset(feature_dict["input_ids"],
                      feature_dict["attention_masks"],
                      feature_dict["labels"])


In [ ]:
# Models to use for sequence cclassification method: "xlm-roberta-base" (implemented here)
# Models to use for embedding-based method: "bert-base" + transformer layer stack (initialized randomly)
# Datasets:
#   English-only: DailyDialogue
#   Multilingual: Opensubtitles (requires splitting into turns, cleaning; but seems overall promising)
#       Model: xlm-roberta


class ModelTrainer(object):

  """ Trains and evaluates a dialogue-continuation classifier. """

  def __init__(self, config, tokenizer, data_processor,
               model_id=0, model_path=None):

    # Instantiate variables
    self.config = config
    self.tokenizer = tokenizer
    self.data_processor = data_processor

    # Initialize model to fine-tune (either from HF or from disc)
    # We use two taget labels, one denoting a valid dialogue continuation and
    # another denoting an invalid one
    if model_path is None:
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.config.checkpoint, num_labels=2)
    else:
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_path)

    # Check if GPU can be used
    self.device = "cuda" if torch.cuda.is_available() and not self.config.use_cpu else "cpu"
    if self.device == "cuda":
      print("CUDA is available, using GPU for model training and evaluation.")
    else:
      print("CUDA is unavailable, using CPU for model training and evaluation.")
    self.model.to(self.device)

    # Define model name
    self.model_name = "next_sentence_cls_{}".format(model_id)  # for multiple runs

    # Create dataloaders for training and validation
    self.train_dataloader = self.data_processor.train_dataloader
    self.eval_dataloader = self.data_processor.eval_dataloader
    # Get test data for testing
    self.test_data = self.data_processor.test_data

    # Model saving path
    self.root_dir = "/content/"
    self.model_save_path = "/content/train_run_{:d}".format(model_id)
    if not os.path.isdir(self.model_save_path):
          os.mkdir(self.model_save_path)


  def train(self):

    """ Trains the model. """

    # Compute number of training epochs from total training steps
    num_train_epochs = max(1, int(np.round(self.config.num_train_steps // max(
        1, (len(self.train_dataloader)) // self.config.gradient_accumulation_steps))))

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in self.model.named_parameters() if not any(
            nd in n for nd in no_decay)],
          'weight_decay': self.config.weight_decay},
        {'params': [p for n, p in self.model.named_parameters() if any(
            nd in n for nd in no_decay)],
          'weight_decay': 0.0}]
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters,
                                  lr=self.config.learning_rate,
                                  eps=self.config.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=1,
        num_training_steps=self.config.num_train_steps)

    # Initialize traning variables
    step, global_step = 0, 0
    tr_loss, last_reported_loss = 0.0, 0.0
    mean_training_loss = 0.0
    best_eval_acc = 0.
    stale_count = 0
    # Reset model gradient
    self.model.zero_grad()

    print("Starting training model: {:s}".format(self.model_name))

    # Report hyper-parameters used during training
    print('=' * 20)
    print('Hyperparameters:')
    print("\tmax_seq_len: {}".format(self.config.max_seq_len))
    print("\tgradient_accumulation_steps: {}".format(
        self.config.gradient_accumulation_steps))
    print("\tnum_train_steps: {}".format(self.config.num_train_steps))
    print("\tlearning_rate: {}".format(self.config.learning_rate))
    print("\tweight_decay: {}".format(self.config.weight_decay))
    print("\tadam_epsilon: {}".format(self.config.adam_epsilon))
    print("\tmax_grad_norm: {}".format(self.config.max_grad_norm))
    print("\ttrain_batch_size: {}".format(self.config.train_batch_size))
    print("\teval_batch_size: {}".format(self.config.eval_batch_size))
    print('=' * 20)

    train_iterator = trange(num_train_epochs, desc="Epoch")
    for epoch_id, _ in enumerate(train_iterator):
      epoch_iterator = tqdm(self.train_dataloader, desc="Iteration")
      for batch_id, batch in enumerate(epoch_iterator):

        # Training mode
        self.model.train()
        batch_inputs = {k: t.to(self.device) for k, t in batch.items()}
        outputs = self.model(**batch_inputs)

        # Compute model loss
        loss = outputs.loss

        # Accumulate loss for gradient accumulation
        loss = loss / self.config.gradient_accumulation_steps

        # Track training loss
        tr_loss += loss.item()

        # Get gradients
        loss.backward()
        step += 1

        # Update model after N gradient accumulation steps
        if step % self.config.gradient_accumulation_steps == 0:
          # Backpropagate, reglarize, optimize
          torch.nn.utils.clip_grad_norm_(
              self.model.parameters(), self.config.max_grad_norm)
          optimizer.step()
          scheduler.step()
          # Reset gradient
          self.model.zero_grad()
          global_step += 1

          # Report
          if global_step % self.config.logging_steps == 0:
            curr_mean_loss = (
                tr_loss - last_reported_loss) / self.config.logging_steps
            curr_learning_rate = scheduler.get_last_lr()
            last_reported_loss = tr_loss
            print("\tGlobal step: {:d} | LR: {} | Avg. loss: {:.3f}".format(
                global_step, curr_learning_rate, curr_mean_loss))

          # Evaluate
          if global_step % self.config.eval_after_steps == 0:
            print("=" * 20)
            print("\tEvaluating on valiadtion set!")
            eval_acc = self.eval()
            if eval_acc > best_eval_acc:
              best_eval_acc = eval_acc
              stale_count = 0
              # Save best model
              self._save_model()
            else:
              stale_count += 1
            print("\tEvaluation accuracy: {:.3f} | Stale count: {:d}".format(
                eval_acc, stale_count))
            print("=" * 20)
            if stale_count >= self.config.patience:
              print("Early stop!")
              break

        # Stop iteration after maximum number of training steps
        if self.config.num_train_steps < global_step:
          epoch_iterator.close()
          break

      # Stop iteration after maximum number of training steps
      if 0 < self.config.num_train_steps < global_step or stale_count >= self.config.patience:
        train_iterator.close()
        mean_training_loss = tr_loss / global_step
        print("== Finished training model: {:s} ==".format(self.model_name))
        print("\tMean training loss: {:.3f}".format(mean_training_loss))
        break

    # Report mean training loss for weighted logit combination
    return global_step, mean_training_loss


  def eval(self):

    """ Evaluates the model. """

    # Track performance
    eval_step = 0
    eval_acc = 0.

    # Iterate over evaluation data
    for batch_id, batch in enumerate(
        tqdm(self.eval_dataloader, desc="Evaluating")):

      # Evaluation mode
      self.model.eval()

      # Iterate over evaluation data
      with torch.no_grad():
        # Prepare inputs
        batch_inputs = {
            "input_ids": batch["input_ids"],
            "attention_mask": batch["attention_mask"]
            }
        labels = batch["labels"].to(self.device)

        batch_inputs = {k: t.to(self.device) for k, t in batch_inputs.items()}
        # Get outputs
        outputs = self.model(**batch_inputs)
        # Get model loss
        logits = outputs[0]

      # Compute model accuracy
      batch_acc = self._compute_accuracy(logits, labels)
      print(batch_acc)
      eval_acc += batch_acc
      eval_step += 1

    # Report model accuracy on the evaluation set
    mean_eval_acc = eval_acc / eval_step
    print("== Finished evaluating model: {:s} ==".format(self.model_name))
    print("\tMean evaluation acc: {:.3f}".format(mean_eval_acc))

    return mean_eval_acc


  def test(self, context=None, targets=None, label=None):

    """ Tests the model on some arbitrary context-target pair.
      In case multiple continuations are marked as valid, returns the one that
      the model is most confident in. """

    # Prepare data for input to model
    # If no data was provided by user, sample from test data

    if context is None or targets is None:
      print("Sampling a test item from the test data!")
      test_sample = random.choice(self.test_data)
      context = test_sample["context"]
      targets = test_sample["targets"]
      label = test_sample["label"]
    else:
      if type(context) == list:
        context = " ".join([c.strip() for c in context])

    print("Context: {}".format(context))
    print("Targets: {}".format(targets))
    print("Expected output: {}".format(label))

    # Create test samples
    test_samples = list()
    for tgt in targets:
      test_samples.append({"context": context,
                           "target": tgt,
                           "label": int(tgt == label)})
    test_samples = self.data_processor.encode_truncate_pad(test_samples)


    # Evaluation mode
    self.model.eval()

    out_id = None
    max_conf = 0.
    # Iterate over continuations
    for smp_id, smp in enumerate(test_samples):

      with torch.no_grad():
        # Prepare inputs
        input_ids = torch.tensor([smp["input_ids"]], dtype=torch.long)
        attention_mask = torch.tensor([smp["attention_mask"]], dtype=torch.long)
        batch_inputs = {
          "input_ids": input_ids,
          "attention_mask": attention_mask
          }
        batch_inputs = {k: t.to(self.device) for k, t in batch_inputs.items()}
        # Get outputs
        outputs = self.model(**batch_inputs)
        # Get model loss
        logits = outputs[0]
        # Get model prediction and confidence
        probs = torch.nn.functional.softmax(logits, dim=-1)
        conf, cls = torch.max(probs, -1)
        if cls == 1:
          if conf > max_conf:
            out_id = smp_id
            max_conf = conf

    if out_id is None:
      return -1

    else:
      return targets[out_id]


  @staticmethod
  def _compute_accuracy(preds, labels):

    """ Helper for computing evaluation accuracy """

    return (torch.argmax(preds, -1) == labels).float().mean().item()


  def _save_model(self):

    """ Saves a model checkpoint to disk. """

    # Save model
    model = self.model.module if hasattr(self.model, 'module') else self.model
    model.save_pretrained(self.model_save_path)



In [ ]:
# Initialize config
config = Config()

# Set random seed
random.seed(config.random_seed)
torch.manual_seed(config.random_seed)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(config.random_seed)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Prepare data
data_processor = DataProcessor(config, tokenizer)

Dowloading data ...
Transforming data ...
Transforming data ...
Transforming data ...
Encoding data ...


Token indices sequence length is longer than the specified maximum sequence length for this model (785 > 512). Running this sequence through the model will result in indexing errors


Encoding data ...


In [ ]:
# Reset GPU
torch.cuda.empty_cache()

In [ ]:
trainer = ModelTrainer(config, tokenizer, data_processor)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at facebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CUDA is available, using GPU for model training and evaluation.


In [ ]:
# Train model
trainer.train()


Starting training model: next_sentence_cls_0
Hyperparameters:
	max_seq_len: 256
	gradient_accumulation_steps: 4
	num_train_steps: 1000
	learning_rate: 1e-05
	weight_decay: 0.01
	adam_epsilon: 1e-08
	max_grad_norm: 1.0
	train_batch_size: 32
	eval_batch_size: 32


Iteration:  14%|█▍        | 100/695 [02:06<12:34,  1.27s/it]

	Global step: 25 | LR: [9.75975975975976e-06, 9.75975975975976e-06] | Avg. loss: 0.697



Iteration:  29%|██▊       | 199/695 [04:11<10:26,  1.26s/it]

	Global step: 50 | LR: [9.50950950950951e-06, 9.50950950950951e-06] | Avg. loss: 0.683
	Evaluating on valiadtion set!


...

0.75
1.0
== Finished evaluating model: next_sentence_cls_0 ==
	Mean evaluation acc: 0.881




Iteration:  89%|████████▉ | 620/695 [14:23<09:14,  7.39s/it]

	Evaluation accuracy: 0.881 | Stale count: 0



Epoch: 100%|██████████| 5/5 [1:18:48<00:00, 945.63s/it]


(868, 0.0)

In [ ]:
# Test model
trainer.test()


Sampling a test item from the test data!
Context: Person 1: It seems that nobody in your class likes Paul . Person 2: The boy is good for nothing . Person 1: As a teacher , I don ’ t know how to help him . Person 2: 
Targets: [" Well , you're right , the resume is limited in how much it can tell someone about a person . That's why job interviews are important to let people know the real you that they can't see from a piece of paper . But resumes can be helpful in explaining things and giving a good impression to a potential employer . ", " I love it . It's so festive ! And where did you get that cup ? It's got the same logo . ", " I'm sorry , dad . But I would rather stay at school than go to cram school . ", ' No , thank you . ', ' That looks great . Do you have ties here ? ', " Yes , I have , Mr . Thomas . We'll pack them two dozen to one carton , and the gross weight is around 25 kilos a carton . "]
Expected output: -1
Encoding data ...


-1

In [ ]:
context = ["Person 1: Do you happen to have their email or phone number so we get in touch?", "Person 2: Email of who? Myself or the store owner ?", "Person 1:"]
targets = ["I dont want to answer.",
           "I dont know.",
           "Email.",
           "Sure, the email is example@companyname.com.",
           "No.",
           "Yes.",
           "Maybe.",
           "Something not understood.",
           "Phone number.",
           "I'm not entirely confident if I'm permitted to share contacts."]
label = -1

trainer.test(context=context, targets=targets)

Context: Person 1: Do you happen to have their email or phone number so we get in touch? Person 2: Email of who? Myself or the store owner ? Person 1:
Targets: ['I dont want to answer.', 'I dont know.', 'Email.', 'Sure, the email is example@companyname.com.', 'No.', 'Yes.', 'Maybe.', 'Something not understood.', 'Phone number.', "I'm not entirely confident if I'm permitted to share contacts."]
Expected output: None
Encoding data ...


'Phone number.'